# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [9]:
# Imports here

import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as Function
from torch import nn, optim
from torch.autograd import Variable
from PIL import Image 
import json
from workspace_utils import active_session
from workspace_utils import keep_awake

%matplotlib inline
%config InlineBackend.figure_format='retina'

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [10]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [11]:
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
# TODO: Define your transforms for the training, validation, and testing sets
data_transforms_one = transforms.Compose([
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])


data_transforms_two = transforms.Compose([
                                    transforms.RandomHorizontalFlip(),
                                    transforms.RandomVerticalFlip(),
                                    transforms.Resize(255),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

data_transforms_three = transforms.Compose([
                                    transforms.RandomHorizontalFlip(),
                                    transforms.RandomVerticalFlip(),
                                    transforms.Resize(255),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder

train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms_one)
valid_dataset = datasets.ImageFolder(valid_dir, transform=data_transforms_one)
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms_one)

# TODO: Using the image datasets and the trainforms, define the dataloaders
#for pre trained models:batch size 64 is suitable
train_loaders =  torch.utils.data.DataLoader(train_dataset, shuffle = True, batch_size = 64)
valid_loaders =  torch.utils.data.DataLoader(valid_dataset, batch_size = 64)
test_loaders =  torch.utils.data.DataLoader(test_dataset, batch_size = 30)


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [12]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [13]:
# TODO: Build and train your network
import torch
import torchvision
import torchvision.models as models
from torch import nn, optim

class Feedforward(torch.nn.Module):
        def __init__(self, input_size, hidden_size, drop_p=0.2):
            super(Feedforward, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(self.hidden_size,102)
            self.dropout = nn.Dropout(p=drop_p)
            self.activation = torch.nn.LogSoftmax(dim=1)
            
        def forward(self,x):
            
            hidden = self.fc1(x)
            relu = self.relu(hidden)
            output_one = self.fc2(relu)
            output_final = self.activation(output_one)
            return output_final
        
        
model = models.vgg13(pretrained=True)

# Start the script and create a tensor
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


model.classifier = Feedforward(25088, 2048)
model = model.to(device)
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01)       
print(model)        
print(model.state_dict().keys())


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): ReLU(inplace)
    (17): Conv2d

In [14]:
# TODO: Build and train your network


In [15]:
from workspace_utils import active_session 
from workspace_utils import keep_awake

steps_taken = 0
running_loss = 0.0
print_step = 10
epochs = 5

with active_session():
    
    for epoch in range(epochs):
        for inputs, labels in train_loaders: 
            steps_taken += 1
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            log = model.forward(inputs)
            loss = criterion(log, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            if steps_taken % print_step == 0:
                loss_v = 0
                accuracy = 0
                model.eval()
                
                
                with torch.no_grad():
                    
                    for inputs, labels in valid_loaders:
                        inputs, labels = inputs.to(device), labels.to(device)
                        log = model.forward(inputs)
                        batch_loss = criterion(log, labels)
                        loss_v += batch_loss.item()
                        
                        #https://towardsdatascience.com/how-to-train-an-image-classifier-in-pytorch-and-
                        #use-it-to-perform-basic-inference-on-single-images-99465a1e9bf5
                        #check accuracy 
                         
                        ps = torch.exp(log)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                        training_loss = running_loss/print_step
                        valid_loss = loss_v/len(valid_loaders)
                        valid_acc = accuracy/len(valid_loaders)
                        
                    print("current epoch: {}/{}". format(epoch+1, 5))
                    print("Training loss: {}".format(training_loss))
                    print("Validation loss: {}".format(valid_loss))
                    print("Validation accuracy: {}".format(valid_acc))
                    print('Training process has finished.')
            running_loss = 0
            model.train()
                    


current epoch: 1/5
Training loss: 0.4332751274108887
Validation loss: 4.335990704022921
Validation accuracy: 0.07966346150407425
Training process has finished.
current epoch: 1/5
Training loss: 0.3855114936828613
Validation loss: 3.802050627194918
Validation accuracy: 0.18865384619969588
Training process has finished.
current epoch: 1/5
Training loss: 0.34907944202423097
Validation loss: 3.593551287284264
Validation accuracy: 0.21908653842715117
Training process has finished.
current epoch: 1/5
Training loss: 0.31040163040161134
Validation loss: 3.000847660578214
Validation accuracy: 0.3620673074172093
Training process has finished.
current epoch: 1/5
Training loss: 0.27125651836395265
Validation loss: 2.5249069929122925
Validation accuracy: 0.4013461539378533
Training process has finished.
current epoch: 1/5
Training loss: 0.256967830657959
Validation loss: 2.4892975367032566
Validation accuracy: 0.4490865377279428
Training process has finished.
current epoch: 1/5
Training loss: 0.179

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [16]:
# TODO: Do validation on the test set

def check_accuracy(test_loaders):
    model.to(device)
    with torch.no_grad():
        accuracy = 0
        model.eval()

        
        for inputs, labels in test_loaders:
            inputs, labels = inputs.to(device),labels.to(device)
            output = model.forward(inputs)
            batch_loss = criterion(output, labels)
            
            #https://towardsdatascience.com/how-to-train-an-image-classifier-in-pytorch-and-
            #use-it-to-perform-basic-inference-on-single-images-99465a1e9bf5
            #check accuracy 
            
            ps = torch.exp(output)
            top, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            fin_test_acc = (100 * accuracy/len(test_loaders))
            
        print("test accuracy: {}".format(fin_test_acc)) 
            


check_accuracy(test_loaders)

test accuracy: 82.02380899872098


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [22]:
# TODO: Save the checkpoint
model.class_to_idx = train_dataset.class_to_idx
torch.save({'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict(), 'arch': 'vgg13' , 'hidden_size': 512, 'epochs':5, 'class_to_idx': model.class_to_idx, 'classifier': model.classifier, 'input_layer' : 25088, 'lr' : 0.01}, 'checkpoint.pth')
model.to(device)

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Feedforward. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): ReLU(inplace)
    (17): Conv2d

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [1]:
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as Function
from torch import nn, optim
from torch.autograd import Variable
# TODO: Write a function that loads a checkpoint and rebuilds the model

def load_chkpnt(path):
    
    checkpoint = torch.load(path)
    model = models.vgg13(pretrained=True)
    

    arch = checkpoint['arch']
    epoch = checkpoint['epochs']
    hidden_layer = checkpoint['hidden_size']
    model.classifier = checkpoint['classifier']
    model = model.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']
    

    optimizer = checkpoint['optimizer']
    
    return model
    


load_chkpnt('checkpoint.pth')

AttributeError: Can't get attribute 'Feedforward' on <module '__main__'>

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
#import numpy as np

def process_image(image):
  
    img_pil = Image.open(image)
    img_transforms =  transforms.Compose([
                                        transforms.RandomHorizontalFlip(),
                                        transforms.Resize(256),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

    image = img_transforms(img_pil)
    return image



image = process_image("flowers/test/1/image_06764.jpg") 
print(image.shape)

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

imshow(process_image("flowers/test/1/image_06764.jpg"))

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.'''
    # TODO: Implement the code to predict the class from an image file
    
    
    model = load_chkpnt('checkpoint.pth')
    device_used = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    if torch.cuda.is_available():
        model.cuda()

    model.to(device_used)
    model.eval()
    
    with torch.no_grad():
        
        image_tensor = process_image(image_path)
        image_tensor = image_tensor.float()
        image_tensor = image_tensor.unsqueeze_(0)

        input = Variable(image_tensor)
        input = input.to(device)
        output = model.forward(input)
        output = torch.exp(output)
        
        probs, class_tp_idx = output.topk(topk, dim=1)
        
        for k in model.class_to_idx:
            class_idx_convert = {model.class_to_idx[k]:k}
            
        for j in class_tp_idx.cpu().numpy()[0]:
            classes = class_idx_convert[i]
            
        top_probability = probs.cpu().numpy()[0]
            
    return top_probability, classes



image_path = 'flowers/test/16/image_06670.jpg'

probability, classes_top = predict(image_path, 'checkpoint.pth')
print(probability)
print(classes_top)
    
    

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
image = Image.open('flowers/test/16/image_06670.jpg')

class_div = []

for k in classes:
    class_div.append(cat_to_name[k])
    
probability, classes = predict(image, 'checkpoint.pth')
    
figure = plt.figure(figsize = [10, 5])
axes_one = plt.subplot(2,1,1)
axes_two = plt.subplot(2,1,2)
axes_one.set_axis_off()
axes_two.set_axis_off()

posit_y = np.arange(5)
plt.barh(posit_y, probability, align='center', color = 'maroon', width = 0.6)
axes_two.set_xlabel('Probability')
axes_one.set_xlabel('classes')
plt.yticks(posit_y)

imshow(process_image("flowers/test/1/image_06764.jpg"))
plt.show()